In [1]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
import mysql.connector
import pandas as pd
import json

In [3]:
def open_database():
    con = mysql.connector.connect(user='dbadmin', password='washywashy',host='127.0.0.1',database='tests')
    return con, con.cursor()

def read_csv(filename):
    df = pd.read_csv(filename, low_memory=False)
    return df

def select_columns(df, columns):
    return df[columns]

def get_data():
    df = read_csv('./data/movies_imbd.csv')
    return df

def get_column_df(col):
    df = read_csv('./data/movies_imbd.csv')
    df = select_columns(df,[col])
    df = df[~pd.isnull(df[col])]
    df.fillna("",inplace=True)
    return df

def from_key_to_key(s, fk, tk):
    fp = s.find("'"+fk+"':")
    tp = s.find("'"+tk+"':")    
    return s[fp+len(fk)+4:tp-2]

def get_single_id_name(s):
    return from_key_to_key(s, "id", "name"), from_key_to_key(s, "name", "poster_path")

In [4]:
# Production Countries

def get_production_countries_data():
    df = get_column_df('production_countries')
    production_countries = {}
    for c in df.values:
        if (c[0] != '[]'):
            lines = c[0].split('},')
            for l in lines:
                l = l.replace("[{'iso_3166_1","{'iso_3166_1").replace("}]","}")

                if l[-1] != "}":
                    l = l + "}"
                l = l.strip()

                fp = l.find("'"+'iso_3166_1'+"':")
                tp = l.find(""+'name'+"")
                initials = l[fp+len('iso_3166_1')+5:tp-4]

    #           getting country was different
                fp = l.find("'"+'name'+"':")
                tp = l.find(""+'}'+"")
                country = l[fp+len('name')+5:tp-1]

                production_countries[initials] = country

    return production_countries   

def create_production_countries_table():
    con, cursor = open_database()
    cursor.execute("drop table if exists tests.production_countries")
    create_table_sql = """
        create table tests.production_countries (
            id int not null auto_increment,
            initials varchar(10),
            name varchar(250),
            primary key(id)
        )
        """
    cursor.execute(create_table_sql)
    con.commit()    
    con.close()
    
def insert_into_production_countries():
    pc = get_production_countries_data()
    values = []
    for key in pc:
        values.append((key,pc[key]))
    con, cursor = open_database()
    cursor.executemany("insert into tests.production_countries (initials, name) values (%s, %s)", values)
    con.commit()    
    con.close()        

In [5]:
# create_production_countries_table()
# insert_into_production_countries()

In [6]:
# Genres 

def get_genres_data():
    genres={}
    df = get_column_df('genres')
    for s in df.values:
            if (s[0] != '[]'):
                parts = s[0].split('},')        
                for p in parts:
                    p = p.replace("[{'id","{'id").replace(" {'id'", "{'id'").replace("}]", "}").replace("} ", "}")
                    p = p.replace("[{'name","{'name")
                    if p[-1] != '}':
                        p = p + "}"
                    p = p.replace("'", '"')
                    obj = json.loads(p)

                    id = obj['id']
                    name = obj['name']
                    genres[id] = name
                    print(id, name)
    print(genres)

def create_table_genres():
    con, cursor = open_database()
    cursor.execute("drop table if exists tests.genres")
    create_table_sql = """
    create table tests.genres (
        id int,
        name varchar(512),
        primary key(id)
    )
    """
    cursor.execute(create_table_sql)
    con.commit()    
    con.close()

def insert_into_genres():
    gens = get_genres_data()
    values = []
    for key in gens:
        values.append((key,gens[key]))
    con, cursor = open_database()
    cursor.executemany("insert into tests.genres (id, name) values (%s, %s)", values)
    con.commit()    
    con.close()     

In [7]:
# create_table_genres()
# insert_into_genres()

In [8]:
# Production Companies

def get_production_companies_data():
    production_companies = {}
    df = get_column_df('production_companies')
    for c in df.sample(1000).values:
        if(c[0] != '[]'):
            parts = c[0].split('},')        
            for p in parts:
                p = p.replace("[{'id","{'id").replace(" {'id'", "{'id'").replace("}]", "}").replace("} ", "}")
                p = p.replace("[{'name","{'name")
                if p[-1] != '}':
                    p = p + "}"
                    p = p.replace("{'name': ", "").replace("'id': ", "").replace('}', '')        
                    parts2 = p.split("', ")
                    if len(parts2) == 2:
                        name = parts2[0].strip()[1:]
                        id = parts2[1]
                        production_companies[id] = name
                    else:
                        parts2 = p.split('", ')
                        name = parts2[0].strip()[1:]
                        id = parts2[1]
                        production_companies[id] = name
    return production_companies


def create_production_companies_table():
    con, cursor = open_database()
    cursor.execute("drop table if exists tests.production_companies")
    create_table_sql = """
    create table tests.production_companies (
        id int,
        name varchar(512),
        primary key(id)
    )
    """
    cursor.execute(create_table_sql)
    con.commit()    
    con.close()
    
def insert_production_companies_data():
    pc = get_production_companies_data()
    values = []
    for key in pc:
        values.append((key, pc[key]))
    con, cursor = open_database()
    cursor.executemany("insert into production_companies (id, name) values (%s, %s)", values)
    con.commit()    
    con.close()    

In [9]:
# create_production_companies_table()
# insert_production_companies_data()

In [16]:
# collection Belonging

def get_collection_data():
    df = get_column_df("belongs_to_collection")
    collection_belonging = {}
    for s in df.values:
        s = s[0]
        
        id = from_key_to_key(s,'id','name')        
        name = from_key_to_key(s,'name','poster_path')
        name = name.replace("'", "").replace('"', '')
        if id != '' and name != '':
            collection_belonging[id] = name
        
    return collection_belonging
    
def create_collection_table():
    con, cursor = open_database()
    cursor.execute("drop table if exists tests.movie_collections")
    create_table_sql = """
    create table tests.movie_collections (
        id int,
        name varchar(512),
        primary key(id)
    )
    """
    cursor.execute(create_table_sql)
    con.commit()    
    con.close()

def insert_into_collection_table():
    cb = get_collection_data()
    con, cursor = open_database()
    values = []
    for k in cb:
        values.append((k, cb[k]))
    cursor.executemany("insert into tests.movie_collections (id, name) values (%s, %s)", values)
    con.commit()   
    con.close()    


In [17]:
create_collection_table()
insert_into_collection_table()